In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("thenlper/gte-large")

# Sentences we want to encode. Example:
sentence = "This framework generates embeddings for each input sentence"

# Sentences are encoded by calling model.encode()
embedding = model.encode(sentence, output_value="token_embeddings")

In [ ]:
embedding = model.encode(sentence, output_value="token_embeddings")
embedding.shape

In [ ]:
embedding.mean(axis=0).shape

In [ ]:
w1 = "payment"
w2 = "onion"

# Compute the embeddings
embedding1 = model.encode(w1, output_value="token_embeddings")
embedding2 = model.encode(w2, output_value="token_embeddings")

# Compute the cosine-similarity
from sentence_transformers import SimilarityFunction

cosine_fn = SimilarityFunction.to_similarity_fn("cosine")
cosine_score = cosine_fn(embedding1, embedding2)

print("Cosine-Similarity:", cosine_score)

In [ ]:
embedding2.shape

In [1]:
from report_metrics import *

/Users/mismayil/.pyenv/versions/3.10.14/envs/cs-gen/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
text1 = "Hello world! How are you doing?"

emb1 = get_embedding(text1)

In [11]:
text2 = "Thank you! I am doing well."
emb2 = get_embedding(text2)

In [ ]:
emb2

In [2]:
text3 = "In the heart of the enchanted forest stood a sentient house, its windows like eyes observing the world. It longed to live, to experience life beyond its rooted existence. One moonlit night, a shooting star granted its wish, transforming the house into a towering golem of wood and stone. As it roamed the forest, its window-eyes saw wonders and met creatures it had only dreamed of. With each step, the house-turned-golem learned the essence of life, its walls now filled with memories instead of rooms."
get_words(text3, remove_stopwords=False, dominant_k=5)

['the', 'of', 'its', 'a', 'house']

In [ ]:
dis = compute_sem_dis(emb1, emb2)
dis

In [3]:
text4 = "As the alien spacecraft hovered silently above the city, its metallic surface gleaming in the moonlight, Dr. Eliza Chen peered anxiously through the observatory's window. Her lifetime of research into extraterrestrial life had led to this moment, but nothing could have prepared her for the sight of the massive vessel, easily the size of a house, descending gracefully towards the Earth. With trembling hands, she reached for the radio transmitter, knowing that her next words could determine whether humanity would live or perish. \"Welcome,\" she breathed into the microphone, her voice barely above a whisper, \"We come in peace.\" The ship's response came not in words, but in a brilliant burst of light that bathed the entire planet in a warm, comforting glow."
compute_theme_uniqueness([text3, text4], cluster_distance_threshold=0.5)

[1.0, 1.0]

In [7]:
compute_surprise(text3), compute_surprise(text4), compute_surprise("".join([text3, text4]))

(0.023017248541417762, 0.016241591014377388, 0.019932572159316275)

In [9]:
compute_n_gram_diversity(text3, max_n_gram=3)

[0.6213592233009708, 0.9705882352941176, 1.0]

In [1]:
from report_metrics import *

/Users/mismayil/.pyenv/versions/3.10.14/envs/cs-gen/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
rel_text1 = "automation innovation software hardware engineering robotics programming digital network data"
rel_text2 = "food cooking recipe restaurant chef cuisine ingredient flavor"
unrel_text1 = "ocean laptop basketball poetry mountain refrigerator zebra guitar democracy candle"

In [ ]:
compute_avg_sem_dis(rel_text1), compute_avg_sem_dis(rel_text2), compute_avg_sem_dis(unrel_text1)

In [3]:
compute_avg_sem_dis(rel_text1), compute_avg_sem_dis(rel_text2), compute_avg_sem_dis(unrel_text1)

(0.20274831851323447, 0.15473486483097076, 0.22474116219414605)

In [5]:
get_words(rel_text1), get_words(rel_text2), get_words(unrel_text1)

(['digital',
  'hardware',
  'automation',
  'network',
  'robotic',
  'program',
  'software',
  'engineering',
  'innovation',
  'datum'],
 ['flavor',
  'recipe',
  'chef',
  'restaurant',
  'cooking',
  'ingredient',
  'cuisine',
  'food'],
 ['mountain',
  'laptop',
  'basketball',
  'zebra',
  'refrigerator',
  'guitar',
  'democracy',
  'ocean',
  'candle',
  'poetry'])

In [6]:
emb1 = get_embedding("democracy")
emb2 = get_embedding("candle")

compute_sem_dis(emb1, emb2)

0.24128174781799316

In [10]:
compute_theme_uniqueness([rel_text1, rel_text2, rel_text2, unrel_text1], cluster_distance_threshold=0.5)

[1.0, 0.5, 0.5, 1.0]

## Update existing metrics

In [8]:
from utils import read_json, write_json, find_files
from statistics import mean
from metrics import compute_surprise
from tqdm import tqdm

data_dir = "../experiments/reports/pilot/run1_report2/gpt-4"
files = find_files(data_dir, "json")

for results_file in tqdm(files):
    results = read_json(results_file)
    if "data" in results:
        config = results["metadata"]["config"]
        preprocessing_args = {
            "lower": config["lower"],
            "remove_punct": config["remove_punct"],
            "remove_stopwords": config["remove_stopwords"],
            "lemmatize": config["lemmatize"],
            "dominant_k": config["dominant_k"],
            "unique": config["unique"]
        }
        results["metrics"]["avg_raw_surprises"] = [mean([result["metrics"]["raw_surprises"][surprise_idx] for result in results["data"] if "metrics" in result and surprise_idx < len(result["metrics"]["raw_surprises"])]) for surprise_idx in range(max([len(r["metrics"]["raw_surprises"]) for r in results["data"] if "metrics" in r]))]
        # for result in results["data"]:
        #     if result["output"]:
        #         try:
        #             surprise, raw_surprises = compute_surprise(result["output"], config["emb_model"], config["emb_type"], config["distance_fn"], preprocessing_args)
        #             result["metrics"]["raw_surprises"] = raw_surprises
        #         except Exception as e:
        #             print(results_file)
        #             print(result)
        #             raise e
                    

        write_json(results, results_file)

100%|██████████| 4/4 [00:00<00:00, 234.81it/s]


In [6]:
results["metrics"]["top_10_n_grams"]

{'1_gram': {'the': 191,
  'a': 177,
  'of': 85,
  'her': 55,
  'it': 48,
  'letter': 45,
  'she': 43,
  'to': 37,
  'stamp': 35,
  'was': 35},
 '2_gram': {'the letter': 33,
  'of a': 28,
  'the old': 23,
  'old woman': 21,
  'it was': 19,
  'pressed the': 17,
  'onto the': 16,
  'woman her': 15,
  'gnarled like': 15,
  'like the': 15},
 '3_gram': {'the old woman': 20,
  'old woman her': 15,
  'gnarled like the': 15,
  'woman her hands': 14,
  'her hands gnarled': 14,
  'hands gnarled like': 14,
  'like the roots': 14,
  'the roots of': 14,
  'pressed the stamp': 13,
  'stamp onto the': 12},
 '4_gram': {'the old woman her': 14,
  'old woman her hands': 14,
  'woman her hands gnarled': 14,
  'her hands gnarled like': 14,
  'hands gnarled like the': 14,
  'gnarled like the roots': 14,
  'like the roots of': 14,
  'the roots of a': 12,
  'pressed the stamp onto': 11,
  'the stamp onto the': 11},
 '5_gram': {'old woman her hands gnarled': 14,
  'woman her hands gnarled like': 14,
  'her han

In [ ]:
import pandas as pd
import ast

df = pd.read_csv("../experiments/reports/pilot/run1_report2/gpt-4/pilot_gpt-4_run1_report2_metrics_global.csv")

df["metric_corpus_n_gram_diversity"] = df["metric_corpus_n_gram_diversity"].apply(lambda x: ast.literal_eval(x))
df["n_gram"] = [list(range(1, len(df["metric_corpus_n_gram_diversity"][0])+1)) for _ in range(len(df))]
df.explode(["metric_corpus_n_gram_diversity", "n_gram"]).head(10)

In [19]:
df["group_id"].unique()

array(['stamp-letter-send', 'gloom-payment-exist', 'petrol-diesel-pump',
       'organ-empire-comply'], dtype=object)

In [13]:
df["metric_corpus_n_gram_diversity"][0]

[0.23887027285782672,
 0.6509456547761552,
 0.8587164750957854,
 0.9123353293413173,
 0.928605654048874]

In [1]:
import seaborn as sns

sns.set_theme(style="whitegrid")

import matplotlib.pyplot as plt
import pandas as pd

claude_data = pd.read_csv("../experiments/reports/pilot/run1_report1/claude-3-5-sonnet-20240620/pilot_claude-3-5-sonnet-20240620_run1_report1_metrics.csv")
gemini_data = pd.read_csv("../experiments/reports/pilot/run1_report1/gemini-1.5-flash/pilot_gemini-1.5-flash_run1_report1_metrics.csv")
gpt4_data = pd.read_csv("../experiments/reports/pilot/run1_report1/gpt-4/pilot_gpt-4_run1_report1_metrics.csv")
human_data = pd.read_csv("../experiments/reports/pilot/run1_report1/human/pilot_human_run1_report1_metrics.csv")

claude_data["model"] = "claude"
gemini_data["model"] = "gemini"
gpt4_data["model"] = "gpt-4"
human_data["model"] = "human"

data = pd.concat([claude_data, gemini_data, gpt4_data, human_data])

In [4]:
import pandas as pd
from plot_metrics import plot_metrics_n_gram_diversity

report_name = "run1_report2"
files = [f"../experiments/reports/pilot/{report_name}/gpt-4/pilot_gpt-4_{report_name}_metrics_global.csv",
         f"../experiments/reports/pilot/{report_name}/gemini-1.5-flash/pilot_gemini-1.5-flash_{report_name}_metrics_global.csv",
         f"../experiments/reports/pilot/{report_name}/claude-3-5-sonnet-20240620/pilot_claude-3-5-sonnet-20240620_{report_name}_metrics_global.csv",
         f"../experiments/reports/pilot/{report_name}/human/pilot_human_{report_name}_metrics_global.csv"]

metric_lst = []

for file in files:
    df = pd.read_csv(file)
    metric_lst.append(df)

plot_metrics_n_gram_diversity(metric_lst, "figures")

KeyboardInterrupt: 

In [2]:
pd.read_csv("../experiments/reports/pilot/run1_report2/human/pilot_human_run1_report2_metrics_global.csv")

,group_id,model_id,metric_num_samples,metric_avg_length_in_chars,metric_avg_length_in_words,metric_avg_length_in_unique_words,metric_avg_length_in_concepts,metric_avg_length_in_sentences,metric_avg_word_length_in_chars,metric_avg_sentence_length_in_chars,...,metric_avg_surprise,metric_avg_n_gram_diversity,metric_avg_inv_homogen,metric_avg_novelty,metric_avg_theme_uniqueness,metric_corpus_dsi,metric_corpus_n_gram_diversity,metric_num_unique_stories,metric_usage,metric_cost
0,stamp-letter-send,human,25,376.92,71.96,51.68,26.52,4.36,4.093020,92.626571,...,0.042553,"[0.6915104836448943, 0.956038259585784, 0.9929...",0.175739,0.015201,0.84,0.210958,"[0.335, 0.8149074537268635, 0.9734734734734735...",25,"{'input_tokens': 0, 'output_tokens': 0, 'total...","{'input': 0, 'output': 0, 'total': 0}"
1,gloom-payment-exist,human,25,341.84,64.92,48.64,25.52,4.00,4.161530,92.058667,...,0.043166,"[0.7206884850969, 0.9676567310683905, 0.993824...",0.179796,0.010896,0.96,0.213749,"[0.35797665369649806, 0.8348164627363738, 0.97...",25,"{'input_tokens': 0, 'output_tokens': 0, 'total...","{'input': 0, 'output': 0, 'total': 0}"
2,petrol-diesel-pump,human,25,452.24,85.84,60.96,33.28,5.16,4.180437,91.502571,...,0.030483,"[0.6815274535500768, 0.9649025007385326, 0.994...",0.155943,0.009814,0.68,0.213922,"[0.3210239194292908, 0.8194794290512175, 0.971...",25,"{'input_tokens': 0, 'output_tokens': 0, 'total...","{'input': 0, 'output': 0, 'total': 0}"
3,organ-empire-comply,human,25,364.60,67.92,50.60,26.44,3.84,4.286189,96.552000,...,0.021366,"[0.7246199155205226, 0.967924976474475, 0.9940...",0.188341,0.006142,0.80,0.214997,"[0.3532258064516129, 0.8262506724045185, 0.969...",25,"{'input_tokens': 0, 'output_tokens': 0, 'total...","{'input': 0, 'output': 0, 'total': 0}"


In [7]:
from metrics import load_spacy_engine, get_sentences
from spacy import displacy

nlp = load_spacy_engine()
# text = "In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight. Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass. A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance. As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun. And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless light."
text = "Rose shouldn't be forced to exist in this gloom. The UK weather was so depressing. \"Get me on a plane to hotter climes immediately!\" she demanded.  Waving her credit card around she screamed \"Come on people! Allow me to make that payment\"."
sentences = get_sentences(text)

for sentence in sentences:
    doc = nlp(sentence)
    displacy.render(doc, style='dep')

In [13]:
from metrics import load_spacy_engine, get_sentences
from spacy import displacy

nlp = load_spacy_engine()
text = "In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight. Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass. A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance. As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun. And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless light."
sentences = get_sentences(text)

clause_labels = ["conj", "cc", "preconj", "ccomp", "xcomp", "acl", "relcl", "advcl"]

for sentence in sentences:
    doc = nlp(sentence)
    num_clauses = 0
    for token in doc:
        if token.dep_ in clause_labels:
            num_clauses += 1
    print(f"Sentence: {sentence}")
    print(f"Number of clauses: {num_clauses}")

Sentence: In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight.
Number of clauses: 4
Sentence: Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass.
Number of clauses: 5
Sentence: A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance.
Number of clauses: 3
Sentence: As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun.
Number of clauses: 7
Sentence: And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless li

In [23]:
from metrics import load_spacy_engine, get_sentences
from spacy import displacy

nlp = load_spacy_engine()
text = "In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight. Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass. A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance. As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun. And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless light."
sentences = get_sentences(text)

def get_paths(token):
    if not list(token.children):
        return [[token.dep_]]
    paths = []
    for child in token.children:
        child_paths = get_paths(child)
        for path in child_paths:
            paths.append([token.dep_] + path)
    return paths

for sentence in sentences:
    doc = nlp(sentence)
    dep_paths = []
    for token in doc:
        paths = get_paths(token)
        print(f"Token: {token.text}")
        print(f"paths: {paths}")
        # dep_paths.extend(paths)
    break

Token: In
paths: [['prep', 'pobj', 'det'], ['prep', 'pobj', 'prep', 'pobj', 'det'], ['prep', 'pobj', 'prep', 'pobj', 'amod'], ['prep', 'pobj', 'prep', 'pobj', 'punct'], ['prep', 'pobj', 'prep', 'pobj', 'relcl', 'advmod'], ['prep', 'pobj', 'prep', 'pobj', 'relcl', 'nsubj', 'det'], ['prep', 'pobj', 'prep', 'pobj', 'relcl', 'nsubj', 'prep', 'pobj', 'amod'], ['prep', 'pobj', 'prep', 'pobj', 'relcl', 'advmod'], ['prep', 'pobj', 'prep', 'pobj', 'relcl', 'prep', 'pobj', 'det']]
Token: the
paths: [['det']]
Token: heart
paths: [['pobj', 'det'], ['pobj', 'prep', 'pobj', 'det'], ['pobj', 'prep', 'pobj', 'amod'], ['pobj', 'prep', 'pobj', 'punct'], ['pobj', 'prep', 'pobj', 'relcl', 'advmod'], ['pobj', 'prep', 'pobj', 'relcl', 'nsubj', 'det'], ['pobj', 'prep', 'pobj', 'relcl', 'nsubj', 'prep', 'pobj', 'amod'], ['pobj', 'prep', 'pobj', 'relcl', 'advmod'], ['pobj', 'prep', 'pobj', 'relcl', 'prep', 'pobj', 'det']]
Token: of
paths: [['prep', 'pobj', 'det'], ['prep', 'pobj', 'amod'], ['prep', 'pobj', 'pu

In [1]:
from metrics import compute_dependency_complexity

text = "In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight. Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass. A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance. As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun. And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless light."
dep_paths, dep_num_clauses = compute_dependency_complexity(text)

Loading spacy engine: en_core_web_sm


In [3]:
dep_num_clauses

[4, 5, 3, 7, 2]

In [4]:
text = "Rose shouldn't be forced to exist in this gloom. The UK weather was so depressing. \"Get me on a plane to hotter climes immediately!\" she demanded.  Waving her credit card around she screamed \"Come on people! Allow me to make that payment\"."
dep_paths, dep_num_clauses = compute_dependency_complexity(text)

In [5]:
dep_paths

[Counter({('aux',): 2,
          ('nsubjpass',): 1,
          ('neg',): 1,
          ('auxpass',): 1,
          ('ROOT', 'nsubjpass'): 1,
          ('ROOT', 'aux'): 1,
          ('ROOT', 'neg'): 1,
          ('ROOT', 'auxpass'): 1,
          ('ROOT', 'xcomp', 'aux'): 1,
          ('ROOT', 'xcomp', 'prep', 'pobj', 'det'): 1,
          ('ROOT', 'punct'): 1,
          ('xcomp', 'aux'): 1,
          ('xcomp', 'prep', 'pobj', 'det'): 1,
          ('prep', 'pobj', 'det'): 1,
          ('det',): 1,
          ('pobj', 'det'): 1,
          ('punct',): 1}),
 Counter({('det',): 1,
          ('compound',): 1,
          ('nsubj', 'det'): 1,
          ('nsubj', 'compound'): 1,
          ('ROOT', 'nsubj', 'det'): 1,
          ('ROOT', 'nsubj', 'compound'): 1,
          ('ROOT', 'acomp', 'advmod'): 1,
          ('ROOT', 'punct'): 1,
          ('advmod',): 1,
          ('acomp', 'advmod'): 1,
          ('punct',): 1}),
 Counter({('punct',): 3,
          ('ROOT', 'punct'): 3,
          ('dobj',): 2,
   

In [6]:
dep_num_clauses

[1, 0, 1, 0, 2, 1]

In [3]:
from metrics import load_spacy_engine
import benepar, spacy
nlp = load_spacy_engine()
# nlp.add_pipe('benepar', config={'model': 'benepar_en3'})
text = "In the heart of an ancient library, where the dust of forgotten lore hung thick in the air, there existed a tome bound not in leather, but in the very fabric of twilight. Its pages, scribed in an ethereal script, spoke of a realm where shadows held dominion and light dared not trespass. A curious scholar, whose eyes hungered for the secrets of the universe, traced the arcane words with a trembling finger, unwittingly offering a payment of his own shadow to the gloom that hungered for substance. As the last syllable echoed through the silent stacks, his silhouette detached from his feet and merged with the book, leaving him pale and luminous, a being of pure radiance in a world that had forgotten the sun. And so, in the library's deepest corner, the scholar's shadow lived on within the pages, a whisper of darkness in an existence now defined by endless light."
doc = nlp(text)
sent = list(doc.sents)[0]
print(sent._.parse_string)
# (S (NP (NP (DT The) (NN time)) (PP (IN for) (NP (NN action)))) (VP (VBZ is) (ADVP (RB now))) (. .))
# print(sent._.labels)
# ('S',)
# print(list(sent._.children)[0])
# The time for action
# to visualize
# https://brenocon.com/parseviz/

(S (PP (IN In) (NP (NP (DT the) (NN heart)) (PP (IN of) (NP (NP (DT an) (JJ ancient) (NN library)) (, ,) (SBAR (WHADVP (WRB where)) (S (NP (NP (DT the) (NN dust)) (PP (IN of) (NP (VBN forgotten) (NN lore)))) (VP (VBD hung) (ADVP (RB thick)) (PP (IN in) (NP (DT the) (NN air)))))) (, ,))))) (NP (EX there)) (VP (VBD existed) (NP (NP (DT a) (NN tome)) (VP (VBN bound) (PP (RB not) (PP (IN in) (NP (NN leather))) (, ,) (CC but) (PP (IN in) (NP (NP (DT the) (JJ very) (NN fabric)) (PP (IN of) (NP (NN twilight))))))))) (. .))


/Users/mismayil/.pyenv/versions/3.10.14/envs/cs-gen/lib/python3.10/site-packages/torch/distributions/distribution.py:55: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(
